In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install -U imbalanced-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.0/258.0 kB 5.6 MB/s eta 0:00:00
  Attempting uninstall: imbalanced-learn
    Found existing installation: imbalanced-learn 0.10.1
    Uninstalling imbalanced-learn-0.10.1:
      Successfully uninstalled imbalanced-learn-0.10.1


## 1. 데이터 분할 및 대체

▶Orange3

<image src="https://github.com/silverpoodle/data-analysis_python/assets/88484476/20fbd31a-60ac-4ae7-89b4-6a1e41ceac5e" width="700"/>

<br/><br/>


▶ **test & train**

<image src="https://github.com/silverpoodle/data-analysis_python/assets/88484476/c1bdedfc-4a2f-4051-a524-580f78ab5422" width="700"/>

- **모델 평가** : 머신러닝 모델을 학습시키고 난 후, 학습에 사용되지 않은 새로운 데이터에 대한 모델의 예측 능력을 평가

- **일반화 성능 평가** : 모델이 학습 데이터에만 잘 동작하는 것이 아니라, 새로운 데이터에 대해서도 잘 동작하는지 평가

- **과적합 방지**: 모델이 학습 데이터에만 지나치게 적합되어 새로운 데이터에 대해 일반화되지 못하는 과적합(Overfitting) 문제를 방지

- **모델 개선**: 모델의 성능을 개선

<br/><br/>

▶ **타깃 분할**

1.   타깃 변수 불평등 분포
    - 실제 현실 세계에서는 특정 이벤트가 발생하는 빈도가 다름
    - 데이터 수집 과정에서 특정 클래스의 샘플이 다른 클래스의 샘플보다 쉽게 수집
    - 분석 목적에 따라 타겟 변수의 분포가 불균형

2.    타겟 변수 분포 확인

    > **value_counts()**

3.    해결 방법

- `imbalanced_learn 라이브러리`: 불균형한 데이터셋에 대한 샘플링 및 분류 문제 해결을 위한 파이썬 라이브러리.
- `RandomUnderSampler`: 다수 클래스 데이터를 적은 수의 샘플로 언더샘플링하여 클래스 간 균형을 조정하는 방법.
- `RandomOverSampler`: 소수 클래스 데이터를 다수 클래스 데이터 수준으로 오버샘플링하여 클래스 간 균형을 조정하는 방법.


In [3]:
import imblearn
print(imblearn.__version__)

0.12.2


In [4]:
!pip install graphviz

In [5]:
import graphviz
import pandas as pd
import numpy as np

df = pd.read_csv('/content/drive/MyDrive/data-analysis/data-csv/healthcare-dataset-3.csv')

df.shape

(3915, 11)

In [6]:
# dtype을 category로 지정해 줄 변수들을 cols2에 임시 저장
cols2 = ['hypertension', 'heart_disease', 'stroke', 'ever_married_encoded', 'gender_encoded',
         'work_type_encoded', 'residence_type_encoded', 'smoking_status_encoded']


# cols2에 저장된 변수들의 dtype을 category로 변경
df[cols2] = df[cols2].astype('category')

In [7]:
# 타겟변수를 제외한 변수만 data에 저장
data = df.drop(['stroke'], axis=1)

  # 타겟변수만 target에 저장
target = df['stroke']
data.shape

(3915, 10)

In [8]:
df['stroke'].value_counts(dropna=False)

stroke
0    3722
1     193
Name: count, dtype: int64

In [9]:
df['stroke'].value_counts(dropna=False, normalize=True)

stroke
0    0.950702
1    0.049298
Name: proportion, dtype: float64

In [10]:
# RandomUnderSampler를 import하여 사용
from imblearn.under_sampling import RandomUnderSampler

# 타겟 변수의 소수 클래스 및 다수 클래스를 1:3의 비율(=1/3)로 언더샘플링
# sampling_strategy=0.333은 소수 클래스의 비율
# random_state는 재현성을 위한 난수 생성 시드
undersample = RandomUnderSampler(sampling_strategy=0.333, random_state=2)

# RandomUnderSampler를 사용하여 언더샘플링을 적용
# fit_resample() 메서드를 사용하여 data와 target에 언더샘플링을 적용,
# 언더샘플링된 데이터를 data_under, target_under에 저장
data_under, target_under = undersample.fit_resample(data, target)

In [11]:
target_under.value_counts(dropna=True)

stroke
0    579
1    193
Name: count, dtype: int64

In [12]:
target_under.value_counts(dropna=True, normalize=True)

stroke
0    0.75
1    0.25
Name: proportion, dtype: float64

In [14]:
from sklearn.model_selection import train_test_split

# data_under와 target_under를 50:50 비율로 train과 test 데이터로 분할
# test_size=0.5는 데이터를 반으로 나누어 각각의 비율을 50%로 설정
# random_state는 재현성을 위한 난수 생성 시드
# stratify=target_under은 타겟 변수의 클래스 비율을 유지하기 위해 사용
X_train, X_test, y_train, y_test = train_test_split(
    data_under, target_under, test_size=0.5, random_state=42, stratify=target_under)

# 데이터 분할 결과를 출력
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)

X_train shape: (386, 10)
X_test shape: (386, 10)


In [15]:
y_train.value_counts(normalize=True)

stroke
0    0.748705
1    0.251295
Name: proportion, dtype: float64

In [16]:
y_train.value_counts()

stroke
0    289
1     97
Name: count, dtype: int64